In [1]:
pip install linearmodels

     |████████████████████████████████| 1.5 MB 4.1 MB/s 
     |████████████████████████████████| 9.8 MB 46.7 MB/s 
     |████████████████████████████████| 68 kB 6.3 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
from google.colab import files
from sklearn.preprocessing import LabelBinarizer
from statsmodels.api import add_constant
from sklearn.model_selection import train_test_split
import statsmodels.stats.api as sms
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.api import add_constant
import statsmodels.formula.api as smf
from linearmodels.iv import compare

uploaded = files.upload()

Saving DatosEducacion.xlsx to DatosEducacion.xlsx


In [3]:
df = pd.read_excel("DatosEducacion.xlsx")

In [4]:
print("Numero total de observaciones = {}".format(df.shape[0]))
print("Numero total de variables = {}".format(df.shape[1]))

Numero total de observaciones = 668
Numero total de variables = 16


In [5]:
df.head(15)

,DIRECTORIO,P1_DEPARTAMENTO,SECUENCIA_ENCUESTA,NivelEducativoCursado,TipoEstablecimiento,UbicacionColegio,MinutosAColegio,AlimentosEscuela,PagoMatricula,PagoUniforme,PagoUtiles,PagoPension,PagoTransporte,RecibioBeca,Tratamiento,TratamientoFin
0,3013752,25,3,4,1,1,45,0,0,120000,200000,0,80000,0,0.997964,1
1,3003749,25,2,2,1,1,40,1,0,60000,50000,0,0,0,0.997443,1
2,3016628,25,4,2,0,1,5,1,80000,95000,70000,68000,0,0,0.996477,1
3,2998448,25,3,4,1,1,20,0,1000000,400000,1000000,110000,0,0,0.996050,1
4,3008596,25,2,7,1,1,120,0,1232000,0,12000,0,0,0,0.992672,1
5,3008462,25,4,3,1,1,10,0,0,150000,100000,0,0,0,0.992579,1
6,3008753,25,4,3,1,1,5,0,0,300000,300000,0,0,0,0.987130,1
7,3013627,25,3,2,1,2,60,1,0,125000,45000,0,0,0,0.987057,1
8,3003746,25,4,3,1,1,10,1,0,0,20000,0,0,0,0.984057,1
9,3013675,25,3,4,1,1,100,1,0,200000,60000,0,0,0,0.983895,1


In [6]:
df = add_constant(df, has_constant="add")

In [8]:
res_first = IV2SLS(df.TipoEstablecimiento, df[["const", "MinutosAColegio","TratamientoFin","UbicacionColegio","AlimentosEscuela"]], None, None).fit(
    cov_type="unadjusted"
)
print(res_first)
df["TipoEstablecimientoInstrumento"] = df.TipoEstablecimiento - res_first.resids

                             OLS Estimation Summary                            
Dep. Variable:     TipoEstablecimiento   R-squared:                      0.2276
Estimator:                         OLS   Adj. R-squared:                 0.2230
No. Observations:                  668   F-statistic:                    196.88
Date:                 Wed, May 18 2022   P-value (F-stat)                0.0000
Time:                         18:07:44   Distribution:                  chi2(4)
Cov. Estimator:             unadjusted                                         
                                                                               
                                Parameter Estimates                                 
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
const                0.6392     0.0346     18.458     0.0000      0.5713      0.7071
MinutosAColegio     

In [13]:
res_second = IV2SLS(df.NivelEducativoCursado, df[["const"]], df.TipoEstablecimiento, df.MinutosAColegio).fit(
    cov_type="unadjusted"
)
print(res_second)

                            IV-2SLS Estimation Summary                           
Dep. Variable:     NivelEducativoCursado   R-squared:                     -3.3726
Estimator:                       IV-2SLS   Adj. R-squared:                -3.3792
No. Observations:                    668   F-statistic:                    33.363
Date:                   Wed, May 18 2022   P-value (F-stat)                0.0000
Time:                           18:08:54   Distribution:                  chi2(1)
Cov. Estimator:               unadjusted                                         
                                                                                 
                                  Parameter Estimates                                  
                     Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------
const                   9.6583     1.1360     8.5022     0.0000      7.4319     

In [14]:
res_ols = IV2SLS(df.NivelEducativoCursado, df[["const", "TipoEstablecimiento","TratamientoFin"]], None, None).fit(
    cov_type="unadjusted"
)
print(res_ols)

                              OLS Estimation Summary                             
Dep. Variable:     NivelEducativoCursado   R-squared:                      0.0601
Estimator:                           OLS   Adj. R-squared:                 0.0573
No. Observations:                    668   F-statistic:                    42.716
Date:                   Wed, May 18 2022   P-value (F-stat)                0.0000
Time:                           18:09:05   Distribution:                  chi2(2)
Cov. Estimator:               unadjusted                                         
                                                                                 
                                  Parameter Estimates                                  
                     Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------
const                   3.9223     0.1397     28.083     0.0000      3.6485     

In [15]:
from linearmodels.iv import compare
res_direct = IV2SLS(df.NivelEducativoCursado, df[["const", "TipoEstablecimientoInstrumento","TratamientoFin"]], None, None).fit(
    cov_type="unadjusted"
)
print(compare({"OLS": res_ols, "2SLS": res_second, "Direct": res_direct}))

                                                Model Comparison                                                
                                                         OLS                      2SLS                    Direct
----------------------------------------------------------------------------------------------------------------
Dep. Variable                          NivelEducativoCursado     NivelEducativoCursado     NivelEducativoCursado
Estimator                                                OLS                   IV-2SLS                       OLS
No. Observations                                         668                       668                       668
Cov. Est.                                         unadjusted                unadjusted                unadjusted
R-squared                                             0.0601                   -3.3726                    0.3077
Adj. R-squared                                        0.0573                   -3.3792          

In [ ]:
res_second.wu_hausman()

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 156.7434
P-value: 0.0000
Distributed: F(1,665)
WaldTestStatistic, id: 0x7f72bd344310

In [ ]:
res_second.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 157.4505
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f72bd32cb50